# Backpack for Learning Rust

There are many tips and tricks I've been collecting along the way
on my Rust journey. These are bits of knowledge that aren't 
written down or easily found in books or any single place. They're an accumulation
of things I've seen in forums, and StackOverflow posts, and
things buried deep in the official docs.

I'm intending this page to be a "living" post. I'll keep adding
to it as I discover stuff.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Backpack-for-Learning-Rust" data-toc-modified-id="Backpack-for-Learning-Rust-1">Backpack for Learning Rust</a></span><ul class="toc-item"><li><span><a href="#You-will-have-to-read-a-Rust-book,-even-if-you've-been-programming-for-a-long-time" data-toc-modified-id="You-will-have-to-read-a-Rust-book,-even-if-you've-been-programming-for-a-long-time-1.1">You <em>will</em> have to read a Rust book, even if you've been programming for a long time</a></span></li><li><span><a href="#Reduce-friction-managing-dependencies-to-Cargo.toml-with-cargo-edit" data-toc-modified-id="Reduce-friction-managing-dependencies-to-Cargo.toml-with-cargo-edit-1.2">Reduce friction managing dependencies to <code>Cargo.toml</code> with cargo-edit</a></span></li><li><span><a href="#Offline-documentation" data-toc-modified-id="Offline-documentation-1.3">Offline documentation</a></span></li><li><span><a href="#Simple(r)-error-handling" data-toc-modified-id="Simple(r)-error-handling-1.4">Simple(r) error handling</a></span></li><li><span><a href="#Default-build-configuration" data-toc-modified-id="Default-build-configuration-1.5">Default build configuration</a></span></li><li><span><a href="#Use-cargo-fmt-instead-of-rustfmt" data-toc-modified-id="Use-cargo-fmt-instead-of-rustfmt-1.6">Use <code>cargo fmt</code> instead of <code>rustfmt</code></a></span></li><li><span><a href="#Setting-up-logging" data-toc-modified-id="Setting-up-logging-1.7">Setting up logging</a></span></li><li><span><a href="#Command-line-parameters-with-paw" data-toc-modified-id="Command-line-parameters-with-paw-1.8">Command-line parameters with paw</a></span></li></ul></li></ul></div>

## You *will* have to read a Rust book, even if you've been programming for a long time

For those of you with programming experience, especially if you
already know several other programming languages, you might have
become used to picking up a new programming language simply
by skimming the tutorial and then jumping over to the standard
library reference and banging out some code.

Rust is not like that.

You *will* have to read a book, and you *will* struggle with
some concepts until they become clear to you, regardless of how 
much experience you have. "Read the book" might be something
you expect to have to do when you learn a programming language that looks 
very different from the more common "C-like" programming
languages. Some examples of these "unfamiliar" languages might be
Scheme, Haskell, Forth, and so on.
At first glance, Rust syntax seems to have a familiar 
"C-like" syntax and so you might think it will fit neatly 
into your mental box of this-is-how-c-like-languages-work.
But this is an illusion, or at best only a partial truth. Rust
brings together the intersection of 3 aspects that are not
"C-like", but presents them in a syntactical form that 
looks "C-like". This is a trap.

These are the three specific aspects you need to learn 
about with intention and attention:
- The borrow checker
- Expression-based syntax
- Trait-based polymorphism

Of the three, I personally find the borrow checker straighforward
and it presents no trouble. It is annoying as hell (in a good way)
but it seems approachable to me. I know that some other people struggle 
with it, so do give it some respect when diving in. I find
`cargo clippy` to be very useful in describing why code 
makes the borrow checker complain.

The expression-based nature of Rust runs deep; deeper than you 
might think if you only see the syntax through the "C-like" 
lens. Many of the standard enums and types make heavy use of it,
like `Result` and `Option`. There is a clear before-and-after
point during your Rust journey when you will start to embrace
the fluent interface that results from the expression based
syntax, and your journey is much easier after you pick that 
up.

I found Traits the most difficult aspect to assimilate—what I 
mean by that is "be able to write working code".
With traits, there is substantial information you have to actually learn, 
at least
if you want to write actual code. You will need to already be aware of
many of the most common traits (e.g. the ones in the standard 
library), especially for strings and iterators, and you'll need to note which 
traits are implemented in 3rd party libraries you might use. 

Early on, I found out that
only some traits are automatically "imported" and available to use.
For others, you need to explicitly import the *trait* in order 
to employ the *methods* required by that trait, on variables of
types that *implement* that trait.  If you read that sentence and
don't understand what it means: you will have to read a Rust book.

# Don't fear the clones

TBD: do use `.clone()` to solve problems with lifetimes and references

# Dealing with two kinds of strings: `&str` and `String`

This tripped me up quite a bit in the beginning. Saying "2 kinds of strings" really doesn't explain anything. Here's the deal: `&str` isn't really a string, it's a reference to a "string slice". I'm not going to get into what is a slice, but the important part of that statement is the word *reference*. As soon as you have a reference, you will have to deal with *lifetimes* and that's where things can get complicated.

Using `&str` is fine within, say, a single function or an otherwise well-understood scope. But it's very hard to pass identifiers of `&str` around between functions or into closures and that kind of thing. Because of the reference! 

While you're learning, and likely even beyond that, I suggest that when you find trouble, create a new `String` instance *from* the `&str`, using either the `.to_string()` method, or `.to_owned()`. This is pretty much just cloning string data. This allocates a bit of new memory that has no strings attached to anything else, so it's good for: 

- returning from functions
- storing in a `Vec` or `HashMap`, and other containers
- passing to a closure or thread

There is at least one situation where it is better to specify `&str` than `String`: a function definition. You can passing `String` values to a function expecting an `&str` value, because there is built-in support in the `String` type to automatically produce a `&str` that works. That string slice refers back to the underlying memory within the `String` so this is a very efficient and flexible option. Such functions can therefore receive both `&str` and `String` types.

# Don't fear the turbofish `::<>`

When I first started with Rust I was afraid of the turbofish. It seemed so alien, arcane and complex.  It is not so. The turbofish is like a pet someone brought along to the party. You just need a little time to get to know each other.

The turbofish gives you a different way of providing a type annotation.  The most common situation that is going to come up while you're learning is the use of the `.collect()` method on iterators. When you process data using an iterator chain, and eventually you want to make a `Vec` of the results, you'll need `.collect()`. The thing is, `collect` wants you to tell it what kind of container you want. It doesn't just assume you want a `Vec`.

Let's have a look:

```rust
// Imagine "data" is a Vec of String
let results = data.iter()
                .map(|s| s.to_uppercase())
                .collect()
```

This will fail because the compiler wants to you to tell it what kind of container you wish `collect()` to produce.  Easy:

```rust
let results: Vec<String> = data.iter()
                               .map(|s| s.to_uppercase())
                               .collect()
```

This works.

For this example, the compiler will be able to figure out the item of the `Vec` so you don't need to provide that; it only really needed the container type, so this will work:

```rust
let results: Vec<_> = data.iter()
                          .map(|s| s.to_uppercase())
                          .collect()
```

Whither the turbofish? Well, the following is *completely equivalent*:

```rust
let results = data.iter()
                  .map(|s| s.to_uppercase())
                  .collect::<Vec<_>>()
```

Do you see I snuck that `::<Vec<_>>` in there? It's the same type signature as what we had before, it just moved inside the fish. It's like the fish swallowed it!

After you convince yourself that the turbofish is just doing the same thing as the original type signature, you're going to wonder, "What is the point of two ways to give a type?"  Well, it turns out that because Rust is an *expression-based* language, you will frequently find yourself in situations where it is convenient to provide parameters "inline". It's hard to explain, but maybe an example might work.  Here, presented without context, is a real example from some hobby code I worked on:

```rust
  // `templates` is a HashMap<String, String>
  let template_fields = templates
      .get(&cf.template)
      .ok_or_else(|| anyhow!(
          "The specified template {} was not found in the \
          list of available templates: {:?}",
          &t, templates.keys().collect::<Vec<_>>()
      ))?;
```

I'm not going to explain everything, but I just want to point out the call to `.collect()` in the code above. Because of the turbofish, I am able to provide a type to the `.collect()` call. If I could not use the turbofish, I'd have to have done this:

```rust
  // `templates` is a HashMap<String, String>
  let template_fields = templates
      .get(&cf.template)
      .ok_or_else(|| {
          let keys: Vec<_> = templates.keys().collect();
          anyhow!(
              "The specified template {} was not found in the \
              list of available templates: {:?}",
              &t, &keys
      }))?;
```

It's very similar, but less fluid because I needed to generate the keys away from where they're provided to the `anyhow!` macro call.

Ok, maybe my example isn't great. It's just what I happened to have had lying around while I wrote this. The point is: don't be scared of the turbofish, it's harmless.

## Reduce friction managing dependencies to `Cargo.toml` with cargo-edit

Install it with `$ cargo install cargo-edit`. This gives you new cargo subcommands to call in any of your projects when you want to add a new dependency. It automatically adds that dependency to your `Cargo.toml` for you. The great thing is that you don't need to know the latest version string: it figures that out too.

Example:

```shell
$ cargo add fstrings
    Updating 'https://github.com/rust-lang/crates.io-index' index
      Adding fstrings v0.2.3 to dependencies
```

It also supposts upgrading or removing deps. See more at [the repo](https://github.com/killercup/cargo-edit).

## Offline documentation

You can view offline documentation for both your dev project as well as all the Rust project documentation. Offline documentation can be very useful when commuting or travelling, or sitting on the beach.

As long as you have the `rustup` docs component installed, you can immediately open a browser window to view the local Rust documentation there:

```shell
$ rustup doc
```

As for documentation for your project, it can be generated like this, as long as all the dependency crates have already all be downloaded:

```shell
$ cargo doc --open
```
The `--open` parameter will open a browser window for you. Note that this documentation includes not only your own project, but also the docs of all your dependency crates.

## Simple(r) error handling

While you're learning Rust, most of your programs are probably going to be command-line programs, i.e., applications. I strongly suggest you use a crate called `anyhow` to help you deal with error handling. Rust is quite pedantic about types and it can be a frustrating experience trying to figure out exactly how to deal with error types.

If you never want to deal with error types, you could just call `.unwrap()` on every `Result` type; however, this is a bad habit to get into, even when you're just learning. For one thing, it means that later you will have to go back and fix up all the `.unwrap()` calls if you get to the point where you want to clean up your program. But the more unfortunate thing is that you never get to learn how to deal with errors correctly.

This is the dilemma:
- You want to avoid calling `.unwrap()`
- You also want to avoid writing `match` statements every single time some function returns a `Result` type.

What to do? This scenario is described in The Book, where the `?` operator is introduced. Within a function that returns `Result`, if you call another function that returns `Result` you can use `?` to automatically either get the `Ok()` value, or propagate the `Err` up to the caller. That's wordy, but this is all I mean:

```rust
fn my_func() -> Result<(), ...> {
    let x = some_other_func()?;   
    Ok(())
}
```

In the example above, `some_other_func` returns a `Result`. If that `Result` is an `Ok` enum (containing the return value), the inner value will be assigned to `x`; but if the `Result` is an `Err` enum, that will be returned from `my_func` (and `x` will never have been assigned.

This `?` is great, but there's a big problem: the ellipses I used for the error type returned by `my_func` above is quite complicated to deal with if it doesn't match the `Err` type that `some_other_func()` can return. If the two error types are incompatible, the Rust compiler will complain. 

If you use `anyhow`, you won't have to worry about error types. The code will look like this:

```rust
use anyhow::Result;

fn my_func() -> Result<()> {
    let x = some_other_func()?;    
    Ok(())
}
```

If you _don't_ use `anyhow`, and the two error types are incompatible, you will have to "box" the error type:

```rust
use std::err::Error;

fn my_func() -> Result<(), Box<dyn Error>> {
    let x = some_other_func()?;      
    Ok(())
}
```

My suggestion: just use `anyhow`.

Later in your journey when you start making crates for other people to use, it can be useful to use the `thiserror` crate rather than the `anyhow` crate. `thiserror` provides conveniences for creating specific named error types that consumers of your crate can use. Both `anyhow` and `thiserror` are made by the same person, David Tolnay.

One more tip: you can create anyhow-compatible errors from strings using a macro provided by anyhow:

```rust
use anyhow::{Result, anyhow}
//                   ^^^^^^ This is a macro

fn main() -> Result<()> {
    // `anyhow!` creates an `anyhow::Error`
    Err(anyhow!("Something went wrong"))
}
```

produces:

```shell
   Compiling playground v0.0.1 (/playground)
    Finished dev [unoptimized + debuginfo] target(s) in 1.09s
     Running `target/debug/playground`
Error: Something went wrong!
```

## Default build configuration

If you're building on Linux, you can create this file to alter
the default settings for compiling programs:

```toml
# ~/.cargo/config.toml
[target.x86_64-unknown-linux-gnu]
rustflags = [
    "-C", "link-arg=-fuse-ld=lld",
]

[profile.release]
lto = "fat"
codegen-units = 1

[profile.dev.package."*"]
opt-level = 3
```

The `rustflags` setting tells cargo to use the much faster `lld` linker. The default linker is `ld`. By default, `lld` will not be installed; you must install it with `sudo apt install lld`, or whatever your package manager requires.

The `profile.release` section enables full "link time optimization" (LTO) which gives the linker more opportunity for applying performance optimizations. Likewise the `codegen-units` also allows the possibility of additional optimizations. Both of these will be at the expense of compile time so these settings should only be used when building a final release.

The final setting, `profile.dev.package."*"`, tells cargo to build your *dependencies* in release mode, even when you're building your dev package in debug mode. The main benefit of this is that your debug build can still run quite fast, depending on what it's doing. This can sometimes dramatically help performance in interactive applications like games, generative art and similar applications.

Note that if necessary you can override these and any other settings by creating a file `./.cargo/config.toml` inside any of your cargo projects. There are times when you want some or all of your dependencies to also build in debug mode, perhaps for debugging them or even just being able to step through the code in debugger.

## Use `cargo fmt` instead of `rustfmt`

At the time of writing, if you execute code formatting with
`rustfmt`, it's going to use the "2015 Edition" edition 
by default. Since most projects are using the 2018 since,
well, 2018, this doesn't work and so you are forced to 
create a `rustfmt.toml` file like this:

```
# rustfmt.toml
edition = "2018"
max_width = 100
```

This is a pain to set up if you make many new projects, which
is quite common while learning.  You can of course place a
global one in `.config/rustfmt/rustfmt.toml`, but for 
edition tracking specifically, it is much more convenient
to just use the edition value that is already written in
your `Cargo.toml`.  The way to format your code that
will use whatever that is, goes like this:

```
$ cargo fmt
```

(Side note: much of the editor integrations I've seen seem
to want to call `rustfmt` directly. This is unfortunate.)

## Setting up logging

As nice as `println!` is, and believe me I'm a fan, it is pure technical debt that you will later have to pay back by removing it.  A much, much nicer option is to use the logging system which allows you to change the verbosity of the debugging messages.  You can include all of your debugging "print" statements, even in releases, and let users turn on those messages to help troubleshoot what went wrong.

The `log` create provides these macros for producing log messages from your program according to different logging levels. For example:

```rust
use log::*;

fn main() {
    trace!("This will only print at debug level");
    debug!("This will only print at debug level");
    info!("This is info lvl, here's a param: {}", 123);
    warn!("Warning");
    error!("Error level");
}
```

That's pretty much all the `log` crate provides, and it's up to other crates to provide ways to configure the logging level, or where to send logs.

The simplest option I've found so far is the `stderrlog` package. It provides easy configuration options to set up logging, and it plays nicely with `structlog` and `paw`, which I discuss in the next section.

## Command-line parameters with paw

There is a library called [paw]() that makes it very, very easy to add robust support to your CLI applications for command-line parameters. It also integrates with another library `structopt` which allows command-line parameters to be defined declaratively.  Also, in the example below I show also how to set up the `stderrlog` crate from options provided on the command line.

This is how you quickly get set up (4 steps):

1. `$ cargo add paw`
2. `$ cargo add structlog`
3. Now edit your `Cargo.toml` by hand: change the `structlog` entry to this (but use the same version as what you got from the previous command):
```toml
[dependencies]
structopt = { version = "<x.y.z>", features = ["paw"] }
```

4. Edit your `main.rs` file to look something like this:

```rust
use log::*;

#[derive(structopt::StructOpt)]
#[structopt()]
struct Args {
    /// Silence all output
    #[structopt(short = "q", long = "quiet")]
    quiet: bool,
    /// Verbose mode (-v, -vv, -vvv, etc)
    #[structopt(short = "v", long = "verbose", parse(from_occurrences))]
    verbose: usize,
    /// Timestamp (sec, ms, ns, none)
    #[structopt(short = "t", long = "timestamp")]
    ts: Option<stderrlog::Timestamp>,
    /// All your other stuff goes here, e.g.
    #[structopt(parse(from_os_str), short = "o", long = "output-dir")]
    output_dir: Option<std::path::PathBuf>,
}

#[paw::main]
fn main(args: Args) -> {
    stderrlog::new()
        .module(module_path!())
        .quiet(args.quiet)
        .verbosity(args.verbose)
        .timestamp(args.ts.unwrap_or(stderrlog::Timestamp::Off))
        .init()
        .unwrap();

    // All the rest of your code goes here. For example:
    println!("This was the output_dir: {:?}", &args.output_dir);
}
```

Your program can now be called, for example, like this:

```shell
$ myprog -vv --timestamp=ms
```

and you will have millisecond timestamps on your log messages, which will be logged at "warning" level. If you wanted to log at "info" level, you would specify `-vvv`, and so on.